# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [4]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
#     '''
#     Your code here
    
#     提示：可參考以下的Stack Overflow: 
#     https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
#     '''
# Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:

        break

    last_height = new_height

soup = BeautifulSoup(browser.page_source)

### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [5]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [6]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 跌深反彈！汽柴油各升6角、7角　民眾加油動作快

記者吳靜君／台北報導
隨著新冠肺炎 （COVID-19）疫情恐慌稍有降低，帶動國際原油價格，台灣中油今（23）日宣布，自明（24）日凌晨零時起汽、柴油價格每公升各調漲0.6及0.7元。
調整後的參考零售價格分別為92無鉛汽油每公升25.5元、95無鉛汽油每公升27.0元、98無鉛汽油每公升29.0元、超級柴油每公升23.3元。
▲加油站。（圖／記者張一中攝）
台灣中油表示，本週受油國大聯盟(OPEC+)仍在考慮擴大減產及市場對於新型冠狀病毒肺炎的恐慌降低等因素影響，帶動國際原油價格上漲。
本週調價指標7D3B週均價為每桶56.91美元，較前週54.39美元上漲2.52美元，新臺幣兌美元匯率貶值0.14元，國內油價依公式計算漲幅為4.1％，92無鉛汽油及柴油均未超過亞鄰最低價，故汽、柴油應各調漲0.6及0.7元
----------------------------------------------------------------------

[1] 5G服務估3季上路　惠譽示警：頻譜支出電信業財務槓桿恐惡化

▲5G示意圖。（圖／123rf）
記者吳靜君／台北報導
惠譽評等指出，由於鉅額的5G頻譜支出以及為了在2020到2021年之間推出5G服務而伴隨的資本支出增加，可能使台灣電信業者的財務槓桿惡化。
台灣的5G頻譜競標分二階段進行，台灣業者在第一階段競爭激烈，將3.5GHz頻寬的頻譜支出，推高到每MHz人均0.71美元，比義大利的每MHz人均0.40美元貴了 80%，是目前全球最昂貴的 5G 頻譜。第順利完成5G首波釋照的第二階段位置競價，尤其是3.5GHz頻寬的頻段標售，更是使頻譜標金繼續攀高。
惠譽評等預期，台灣大哥大如全部以債務支應新台幣310億元的頻譜支出，則其2020與2021年的營運資金流 (FFO) 調整後淨槓桿率將分別惡化至2.9倍與 2.7倍。
台灣大哥大資本支出預期也會增加到新台幣110億元以便推出5G服務，但是相較2019年估計新台幣71億，卻大幅的增加 。惠譽表示，台灣大哥大若減少發放一年約新台幣150億的股利，或選擇全部或部分出售

▲新冠肺炎疫情增溫，瑞銀認為，大宗商品價格反應過度，不過油價、金價後市看俏。（圖／達志／示意圖）
記者紀佳妘／台北報導
新冠肺炎（COVlD-19）疫情拖累大宗商品價格表現，今年以來價格已下跌至少6%，不過瑞銀（UBS）預估，石油、黃金價格可望隨著疫情穩定後回升，油價可望在2020上半年回升至62美元、金價則上看1,600美元。
瑞銀財富管理投資總監辦公室對大宗商品價格發表最新看法指出，隨著新冠肺炎疫情升溫，今年以來大宗商品價格已下跌6%至7%，其中布蘭特油價自1月中以來一度跌破55美元以下。
不過，瑞銀認為，疫情提高大陸相關資產的風險溢價，特別是大宗商品價格有反應過度的跡象，預期疫情將在本季見頂，但由於需求走軟及部份供應鏈中斷，短期重擊消費、航空等產業，將下修今年亞股成長率預估值，從原先的13.7%降至12.4%，但仍相信下半年獲利會有顯著復甦。
在油價部分，石油輸出國組織及聯盟（OPEC+）加大減產，避免油價進一步下滑，使得布蘭特油價上周連漲4天至57美元。瑞銀預期，隨著疫情得以控制、經濟活動開始復甦，油價在6月底可望逐步回升至62美元。
至於金價部分，在避險需求增溫下，黃金繼續發揮避險作用，有助分散資產組合的風險，瑞銀預估，今年美國經濟成長率走低、債券收益率大幅下降，實際利率保持在低點，讓持有黃金的機會成本因而下跌，加上美元下半年可能走軟，有助於支持金價今年內翻揚，每盎司可望漲至1,600美元。
----------------------------------------------------------------------

[11] 退休理財首重風險管控！　目標日期策略對抗市場波動

▲ 退休理財規劃需要維持穩定步調。（圖／pixabay）
記者李瑞瑾／台北報導
新冠肺炎(COVID-19)疫情打亂市場短線行情，影響投資人看多的信心，不過多數法人機構仍對中長線經濟展望與市場修復力維持正面看待，尤其是全球主要央行延續寬鬆立場，後續仍不排除加碼刺激可能性，資金面仍算寬鬆。富蘭克林華美投信表示，對有長期理財習慣的投資人來說，建議暫勿受短線波動影響，尤其是退休理財策略屬於長期投資，建議維持恆定步調，較能挺得過這一段波動。
富蘭克林華美投信表示，當市場波動提高時，投資人除了想減少退休金投資風險，亦期望維持或提高收益機會，不妨趁早採取接近人生周期概念的「目標

▲未來不論是本國或外國籍航班，起降時機上物品明細都要向海關申報。（圖／記者彭懷玉攝、機內示意圖）
記者吳靜君／台北報導
財政部今（20）日表示，「運輸工具進出口管理辦法」修正預告已經結束，即日起正式上路。修正重點在於未來飛機起降，機上的銷售免稅物品包含菸、酒、物品以及餐飲等、清單明細等都要跟海關申報，包含飛機入境未銷售完成的免稅物品也在內，以防去年國安人員私菸案類似案件重演。
財政部關務署表示，該辦法自今日起開始上路，未依照規定若遭海關抽查的話，可依法處分。
去年爆發國安人員透過總統專機禮遇通關等漏洞，走私私菸案引起社會討論。而海關也針對各項法規積極進行修補，以免類似的案件再重演。針對機上的管理，海關也設置了線上申報系統，也已經開始運作。
關務署為了優化海關對飛機卸載及裝運上機販售、使用機上卸載尚未販售商品的管理，這次修法規範機長或者是運輸業者飛機抵達本國機場卸載機上販售商品像是免稅菸、酒、精品等，應該備妥物品清單明細向海關申報；至裝載上機專供航行國際航線頻繁提用的餐點、飲料（含酒類）及機上販售的商品明細等，亦應於飛機起飛出境前申報。
簡單來說，就是不論是本國的飛機、外國籍飛機，起飛、降落時都要向海關申報機上的販售商品（或未售完商品）、餐點等明細，以讓海關可以掌握機上的物流。
而關務署在去年法規預告期間，也與各運輸業者討論，部分外國籍航班若遇到申報問題，國內的倉儲業者可以代為申報。且目前海關也有線上申報作業，對業者來說阻礙性已經小了很多。關務署表示，因為業者申報明細，日後海關查驗、抽查有依據可循，如果業者有虛報、謊報的情況，海關可依據同辦法（運輸工具進出口管理辦法）81條與86條處分，每架可處6000元到3萬元以下的罰鍰，並且可以按次處罰。
----------------------------------------------------------------------

[21] 北都危機踩煞車？NCC：抗疫徵用緯來、東森頻道不得斷訊

----------------------------------------------------------------------

[22] 東京奧運能否如期舉辦引發關注　日股奧運受惠概念股受驚下跌

▲東京奧運若遭疫情影響，將對日本造成重傷害。（圖／路透社）
記者謝仁傑／綜合報導
日本最大廣告公司，也是東京

文／理周教育學苑．方鈞
Ｑ：自中美貿易戰以來，華為一直是川普想要動刀重點，川普政府利用美司法部起訴華為、遊說盟友勿用華為產品，更在2月16日傳出川普政府正考慮修改「外國直接產品規定」，限制外企使用美國設備替華為製造晶片，試圖切斷中國獲取美國半導體技術的管道，川普封殺令來勢洶洶，對台灣產業鏈會有影響嗎？
Ａ：華為風暴越演越烈，已經從原本國防安全防護演進為供應鏈戰爭，如果「外國直接產品規定」修改通過，未來只要是使用美國半導體設備企業為華為製造晶片，都必須向美國政府取得許可證，由於目前晶片製造商所使用設備多數使用科磊(KLA-Tencor)、應用材料(Applied Materials)、科林研發(Lam Research)等半導體設備製造，中國尚無一條全中國生產的晶片供應鏈，一旦草案通過，無疑將使華為拿不到任何晶片。
華為旗下海思半導體的主要晶片供應商是台積電，是否會影響台積電出貨狀況，未來也將列為觀察重點，外媒報導草案送審約在去年十一月左右，是否通過美國政府官員持保留看法，此舉雖被評定為貿易不平等，但如果藉此遲滯中國半導體業發展，讓美國保有在科技產業龍頭地位，將有利後續中美貿易談判。
雖華為風暴還未來襲，但中美貿易戰及武漢病毒的衝擊對中國經濟已經元氣大傷，全世界金融海嘯會不會由中國開始擴散，投資人還是需小心應對，尤其在狀況不明時，遠離華為概念股，才能夠明哲保身，華為概念股相關的晶圓、封測、網通、手機組裝供應鏈廠商，還是要酌量減少手中持股。
台股高檔震盪，任何國際利空消息都有可能造成股市急速下跌，在這樣的背景下還是建議投資人減少持股比例，空手等待機會較佳。
----------------------------------------------------------------------

[34] Sprint和T-Mobile同意修改合併協議！　軟銀將持股24%、德國電信持股43%成最大股東

▲T-Mobile執行長John Legere和Sprint執行長Marcelo Claure在紐約證券交易所合照。（圖／路透社）
記者謝仁傑／綜合報導
擁有Sprint 80％以上股份的軟銀，取得的換股比例是每股T-Mobile換11股Sprint，軟銀將釋出約4880萬股T-Mobile股份。
交易完成後，軟銀和德國電信將分別持有新公司24％和43％的股份。
T-

▲文曄董事長鄭文宗（左2）與祥碩董事長沈振來（右2）、祥碩總經理林哲偉（右1）。（圖／記者周康玉攝）
記者周康玉／台北報導
IC設計廠祥碩（5269）與半導體通路廠文曄（3036）今（21）日共同宣布要交叉持股合作，雙方以「增資發行新股」方式受讓對方增資發行之新股，每1股祥碩普通股換發19股文曄普通股，股份交換完成後，祥碩將持有文曄約22.39%股權，文曄將持有約13.04%祥碩股權。
文曄董事長鄭文宗表示，這決定不是突然的，原本就跟祥碩公司熟識，一直經常性討論產業走勢及發展，近兩個月決定要合作，真正細部討論是近一個月的事。
文曄增資後，將從5.91億股增加1.7億股，股本提高到7.61億股，擴充幅度約28.76%；祥碩則是從6億股增價至6.9億股，擴充幅度約為15%。此外，文曄增資後，大聯大持有文曄股權將從30%降至22.6%，仍高於祥碩持有文曄的22.39%股份。
祥碩總經理林哲偉表示，母公司華碩對於此合作表示支持，透過換股合作，雙方預計透過策略聯盟扶植具成長潛力的新創IC設計公司，祥碩可分享IC設計開發經驗，文曄從系統整合與市場需求方面提供協助，建立有競爭力的策略聯盟。
鄭文宗表示，文曄過去每隔幾年就會辦理一次增資，也因為增資會減低負債比，雖然股本稀釋後會讓EPS下滑，但因為文曄近年穩定成長，EPS很快就會回歸正軌，形成正向循環，讓公司有更多的籌資空間，也讓股東可保有長期穩定獲利。
對於是否藉由祥碩來制衡大聯大？鄭文宗表示，從大聯大一月底收購完成後，他在公司內部就沒再對此事再發言過，說什麼都不是，乾脆不要說，但和祥碩之間後續合作可能會更密切，不排除讓祥碩加入文曄董事會。
▼文曄董事長鄭文宗（左起）和祥碩總經理林哲偉於重訊後舉行會後記者會。（圖／記者周康玉攝）
----------------------------------------------------------------------

[45] 陸成全球5G手機最大貢獻者！佔總銷量的近50%

▲華為（包括其子品牌榮耀）佔據了全中國大陸5G手機銷售的74%。（圖／路透社）
記者王曉敏／綜合報導
台灣5G競標第二階段今（21）日登場，揭示著我國5G市場正在萌芽，不過此前，5G網路及設備已開始在韓國、美國等國家陸續推出，而根據最新統計顯示，全球5G市場最大的貢獻者是中國大陸，佔整體5G設備銷量的近一

▲富邦人壽首創網銀「電子化授權保險費扣款服務」，提供客戶更貼心的服務體驗。（圖／資料照）
記者紀佳妘／台北報導
富邦人壽今（21）日攜手台北富邦銀行，近期領先業界推出「電子化授權保費扣款服務」，提供保戶免留存印鑑簽章，全年24小時，只要動動手指就可以線上快速完成保險費約定扣款的服務。
富邦人壽總經理陳俊伴表示，隨著數位金融、保險科技不斷創新，富邦人壽除了持續提升業務人員的「數位力」、強化服務效率，更針對客戶投保歷程的各階段進行作業效能優化，以確保保戶權益獲得更完善的保障。
國內數位帳戶近年來出現爆發性成長，2019年底已突破300萬戶，將帶動民眾以數位存款帳戶來約定保險費扣繳的需求，但現行保戶如果想以數位存款帳戶約定保險費扣繳，必須先到銀行留存印鑑簽章，並填寫授權書將印鑑章樣式或簽名樣式送交保險公司受理後，再送交銀行核對印鑑始能完成授權程序，整個申請過程的往返非常耗時、繁複，約要2至8個工作天，加上如保戶不小心蓋錯印鑑章，整個申請流程就要重來，缺乏效率且相當不便。
觀察到此客戶體驗斷點，富邦人壽啟動服務優化專案，攜手台北富邦銀行，領先同業首創網銀「電子化授權保險費扣款服務」，讓保戶免留存印鑑簽章，24小時隨時隨地，透過簡易線上申請機制，即可輕鬆完成保險費約定扣款的服務申請。
不論是一般存款帳戶或數位存款帳戶，若富邦人壽的保戶已有台北富邦銀行的網銀帳號密碼，即可選擇「電子化授權保險費扣款服務」線上約定以台北富邦銀行的存款帳戶扣繳個險保險費。
換句話說，保戶只要動動手指掃描QR Code或點選簡訊的連結網址，輸入台北富邦銀行的網銀帳密認證後，就能即時完成富邦人壽保險費扣款約定，這項創新服務功能，讓傳統等待銀行核對印鑑約需2至8天，大幅降為「0天」。
----------------------------------------------------------------------

[55] 賴香伶質詢處女秀提「不裁員補貼」構想　蘇貞昌回應了

▲台灣民眾黨賴香伶質詢。（圖／記者屠惠剛攝）
記者蔣婕妤／台北報導
立法院新會期21日開議，民眾黨立委賴香伶質詢處女秀主攻新冠肺炎防疫議題。面對行政院長蘇貞昌，賴香伶提問若勞工被指派到疫區工作該如何因應，蘇貞昌回應如果雇主沒有做好安全健康防護措施，勞工有權拒絕到疫區工作。另外，對於賴香伶提出「不裁員補貼」想法，蘇貞昌則強

▲台酒因應防疫中心需求，全力趕製75%防疫酒精。（圖／記者唐詠絮攝）
記者謝仁傑／綜合報導
因應新冠肺炎（COVID-19）疫情持續，為配合全台防疫酒精需求，台酒宣布售價40元的300ml的75％防疫酒精銷售地點，除原本自有門市及全台六千多家健保藥局外，自下周一起亦將陸續供貨至全台4大便利超商銷售，希望能讓更多民眾有酒精使用。
台酒表示，目前台灣在中央防疫指揮中心的運籌帷幄下，新冠肺炎疫情並未擴大，惟考量新冠病毒傳染途徑包括接觸傳染，「勤洗手」及「消毒」是阻擋此波病毒傳播的重要手段，因應配合疫情中心的需求，已責成轄下宜蘭、隆田、屏東及花蓮等酒廠，全力趕製75％防疫酒精。
為便利民眾購買防疫酒精，台酒除於自有門市據點已持續供應75％防疫酒精外，2月17日至22日開始也已擴大配送至全台六千多家健保藥局，預計下周一開始更將進一步擴大供貨至全台4大便利超商銷售，屆時全台有將近2萬的銷售據點供民眾購買75%防疫酒精。
台酒呼籲，針對本次疫情民眾應記得「勤洗手」及「消毒」，為了將有限資源更普及化，也希望大家不要囤積不必搶購，家裡夠用就好，台酒一定會生產足量的防疫酒精供給社會大眾，共同守護台灣。
----------------------------------------------------------------------

[70] 戴口罩，不如啟動「防疫保護罩」！新潤機構翡麗＆鉑麗規劃三重防護　全面守護健康

▲新潤建設機構首創「防疫氧生宅」，將安心健康的居家環境擴大至整體社區。
圖、文／新潤建設提供
新冠肺炎（COVID-19）再度喚起國人對防疫的警覺心，從SARS、MERS、H1N1、H5N1 …到目前全球擴散的武漢肺炎（新型冠狀病毒），以及每年都會奪走數百萬、甚至上千萬性命的流感病毒，病毒變異的速度不斷加速，與其等待研發新疫苗，不如從自身防疫開始做起。「戴口罩」是大家最熟悉的防疫措施，但是口罩只能保護自身安全，如何將保護層面擴大至居家環境健康？新潤建設機構提出「新潤氧生宅」建築新概念，不只能守護居家環境健康、甚至從步入大廳的那一刻起，就啟動防疫保護罩，將防疫守護的範圍擴大至整體社區！
▲「防疫氧生宅」特別著重居家環境健康，透過防霾紗窗、單流新風機的雙重空氣淨化，從源頭阻絕PM2.5、霧霾入侵，取代空氣清淨機。
戶戶均規劃防霾紗窗＋單流新風機，能阻絕霧霾與PM

▲宏碁董事長陳俊聖為防疫站在保護罩後方開講。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
宏碁（2353）董事長陳俊聖今（20）日在汐止總部舉辦新春記者會，但是由於自己擔任台灣商務代表，剛從澳洲雪梨參加完亞太經濟合作會議返台，因此特別站在訂製保護罩演說，小心謹慎地做好防疫措施。
陳俊聖今日出場就站在特別訂製的保護罩後方，並提到自己剛從澳洲參加完亞太經合會返國，因此特別將自己隔離，接著播放《全境擴散》的電影片段，分享以前看這部電影就是想像的畫面，現在看到這部電影已經成為現實的場景。
據悉，這個剛好符合陳俊聖身高的保護罩是壓克力做成，並且是宏碁為了昨日的法人說明會，以及今日的新春記者會，特別在這週請人訂製而成，目前都放在宏碁汐止總部，暫時還沒有其它機會需要使用。
----------------------------------------------------------------------

[84] 收納櫃龍頭樹德加碼投資台灣　斥資逾18億元拚多角化轉型

▲經濟部今（20）日通過收納櫃龍頭樹德企業在台投資案，將斥資超過18億元於南投南崗工業區打造新企業總部。（圖／記者湯興漢攝）
記者林淑慧／台北報導
經濟部今（20）日通過根留台灣企業投資案，收納櫃龍頭樹德企業將斥資超過18億元，在南投南崗工業區擘建具有南島文化特色的新總部「半山夢工廠」，力拚多角化經營在台升級轉型。
官員指出，在許多辦公桌上可見的「樹德櫃」是不折不扣的臺灣之光，曾經奪下德國IF設計獎等多項設計大獎的樹德企業，發跡於臺中家庭代工，歷經半世紀淬鍊走上品牌之路，並以自有品牌研發出全世界第一個塑膠工具箱。
樹德堅持只用臺灣本地產無毒可回收全新原料，將美學、機能、自然共生等理念注入產品細節，為居家、辦公、工業塑造合宜美觀的收納空間，近年也開創3C、醫療新事業，將2大品牌SHUTER、Livinbox插旗全球65國，取得歐美20多項專利。
為了在傳產外找尋多角化經營的轉型出路，樹德斥資逾18億元擘建新總部「半山夢工廠」，汲取美、德、澳建築設計經驗，融匯智慧製造、研發、文化觀光與永續環保於一體，同時將招募73名專業人才。預計完工後將申請美國LEED及臺灣綠建築雙標章，期望帶動南崗工業區廠家一同轉型為零排放、零污染企業，讓臺灣製造能量在國際舞臺上更加發光發熱。
依據經濟部投資台灣事務所統計，「根留臺灣企業

▲金管會主委顧立雄。（圖／記者紀佳妘攝）
記者紀佳妘／台北報導
南山人壽15日公告，董事尹衍樑要求對擔任副董事長、兒子尹崇堯及總經理許妙靜記大過，引起喧然大波。金管會主委顧立雄今（20）日表示，「不符合公司治理原則」，他認為總經理的專業度要拿捏，「大股東再怎麼生氣就跟他say no！」。
金管會主委顧立雄今天在例行記者會上表示，以「不符合公司治理原則」一句話來說明，且已透過保險局請南山寫報告說明。在這種情況下，南山人壽擁有許多保戶，未來要如何獲取保戶的信賴，這是非常重要的事情，此舉已具體反映公司治理受到傷害，對於這樣的傷害該如何彌補，以及對金管會提出什麼樣的承諾，是主管機關最為關切的事項。
顧立雄進一步指出，從這件事情來看，「反映兒子怎麼對抗老爸呢？」不過他認為應思考的是一家公司的副董事長扮演什麼角色，更重要的是董事長要扮演角色，以及總經理專業拿捏，且在董事會應有良善的運作，以及發揮應有的職能，未來要看南山如何符合公司治理的運作，無論做出一個規劃也好、承諾也好，必須要認真看待這件事情。
顧立雄認為，這次事件與先前董事長杜英宗的個人色有很大不同，原先是由小事情引起，大股東車子被撞壞，一時情緒上來才公告懲處，但某種程度來看，從主管機關的立場，要避免大股東因一時興起，對公司治理造成的傷害，相信這次關切會對公司帶來一定程度的警剔。
顧立雄說，「大股東再怎麼生氣就跟他say no！」他強調，重點是在董事會職能應如何有效發揮，且在董事會前，總經理又要如何有專業把持，相信這次關切下會有所改善。
據了解，南山人壽15日內部發出2起懲處公告，首先是南山人壽因業績不佳，連續2年排名業界第三，董事尹衍樑要求副董事長尹崇堯、總經理許妙靜各記大過一次；再者是因停車管理不善，導致董事座車車頂與後方保險桿損傷，一名高層主管被記小過。
----------------------------------------------------------------------

[94] 財訊／台塑王文潮　從石油王子變成抗疫推手

文／林苑卿
武漢肺炎疫情爆發後，讓台塑石化受到外界關注的不是油價議題，竟是一款除菌棒。這是由台塑化與日商日機裝（Nikkiso）合資成立的福機裝公司，利用UV-LED殺菌技術所生產，商品上市4個月以來原本沒沒無聞，沒想到近來隨著武漢疫情爆發，商品也跟著暴紅，甚至賣到缺貨！